<a href="https://colab.research.google.com/github/SeanBarnier/HAFS_Air-Sea/blob/main/HAFSA_ocean-atmosphere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Set up environment

In [ ]:
!pip install cfgrib
!pip install cartopy
!pip install tropycal

In [ ]:
from tropycal import tracks, rain
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt
import cfgrib
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cft

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

User parameters

In [ ]:
name = "Milton"
tcNum = "14"
filepath = f"/content/drive/MyDrive/ColabNotebooks/{name}"
trackType = ""

centralTime = dt(year=2024, month=10, day=7, hour=6) #Time when Milton began its most rapid intensification
daysBefore = 1 #Days before the focal point
daysAfter = 2 #Days after focal point

figureSuffix = "_RI"

Get best track data and find interested point

In [ ]:
bt = pd.read_csv(filepath + "/hurdat2_" + name + trackType + ".csv")

centralTimeStr = centralTime.strftime("%Y-%m-%d %H:%M:%S")
point = (bt[bt.time == centralTimeStr].iloc[0].lat, bt[bt.time == centralTimeStr].iloc[0].lon)
point

Retrieve HAFS-A Data

In [ ]:
dateFormat = "%Y-%m-%d %H:%M:%S"
runFormat = "%Y%m%d%H"

start = centralTime - pd.Timedelta(days=daysBefore)
end = centralTime + pd.Timedelta(days=daysAfter)

Find times needed

In [ ]:
fcastTimes = {} #Key: initiation, item: Forecast Hour

initTime = start
while initTime <= end:
  validTime = initTime
  fcastTimes[initTime] = []

  while validTime <= end:
    fcastTimes[initTime].append(validTime)
    validTime += pd.Timedelta(hours=3)

  initTime += pd.Timedelta(hours=6)

"""for row in tc.iloc:
  fcastTimes[row.time] = []
  rowTime = dt.strptime(row.time, dateFormat)
  if rowTime.hour % 6 != 0 or rowTime.minute != 0: continue #Skip any lines that don't have a HAFS forecast at the same time

  for fhour in range(0, 127, 3):
    valid = rowTime + pd.Timedelta(hours=fhour)
    if valid <= endDT:
      fcastTimes[row.time].append(fhour)"""

In [ ]:
len(fcastTimes)

Get data from HAFS-A output.

In [ ]:
#Data storage structure: Run time, valid time, layer, variable
amtVars = ["T", "q", "u", "v", "gh"]
oceVars = ["T", "s", "u", "v"]

atmFiles = []
atm = {}
oceFiles = []
oce = {}

In [ ]:
atmLayers = [1000, 925, 850, 700, 500] # in hPa
oceLayers = [12.825, 24.895, 48.33, 98.75, 202.150] # In m. Values are directly from file. I don't know what these should be.

In [ ]:
import xarray as xr
oceURL = "https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/20241007/00/14l.2024100700.hfsa.mom6.f000.nc"
oceFile = "oce_20241007_00_f000.nc"
!wget -O {oceFile} {oceURL}
oceData = xr.open_dataset(oceFile, decode_times=False)
oceData

In [ ]:
atmURL = "https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/20241007/12/14l.2024100712.hfsa.storm.atm.f000.grb2"
atmFile = "atm_20241007_12_f000.grb2"
#!wget -O {atmFile} {atmURL}
atmData = xr.open_dataset(atmFile, engine="cfgrib", decode_timedelta=True, filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'isobaricInhPa'})
atmData

In [ ]:
bucket = "https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/"

for init, validList in fcastTimes.items():

  atm[init] = {}
  oce[init] = {}

  initDate, initHour = init.strftime("%Y%m%d_%H").split("_")

  for valid in validList:

    atm[init][valid] = {}
    oce[init][valid] = {}

    fhour = str(int((valid-init).total_seconds() / 3600))
    while len(fhour) < 3: fhour = "0" + fhour

    atmURL = bucket + initDate + "/" + initHour + "/" + tcNum + "l." + initDate + initHour + ".hfsa.storm.atm.f" + fhour + ".grb2"
    atmFile = "atm_" + initDate + "_" + initHour + "_f" + fhour + ".grb2"
    atmFiles.append(atmFile)

    oceURL = bucket + initDate + "/" + initHour + "/" + tcNum + "l." + initDate + initHour + ".hfsa.mom6.f" + fhour + ".nc"
    oceFile = "oce_" + initDate + "_" + initHour + "_f" + fhour + ".nc"
    oceFiles.append(oceFile)

    !wget -O {atmFile} {atmURL}
    !wget -O {oceFile} {oceURL}

    atmData = xr.open_dataset(atmFile, engine="cfgrib", decode_timedelta=True, filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'isobaricInhPa'})
    oceData = xr.open_dataset(oceFile, decode_times=False)

    #Longitude in atm files are in degrees east, but are -180 - 180 in oce files. point has them from -180 - 180
    for layer in atmLayers:
      validPoint = atmData.sel(latitude=point[0], longitude=point[1] + 360, isobaricInhPa=layer, method="nearest")
      atm[init][valid][layer] = {}
      atm[init][valid][layer]["T"] = float(validPoint.t.data)
      atm[init][valid][layer]["q"] = float(validPoint.q.data)
      atm[init][valid][layer]["u"] = float(validPoint.u.data)
      atm[init][valid][layer]["v"] = float(validPoint.v.data)
      atm[init][valid][layer]["gh"] = float(validPoint.gh.data)

    for layer in oceLayers:
      oce[init][valid][layer] = {}
      oce[init][valid][layer]["T"] = float(oceData.sel(yh=point[0], xh=point[1], z_l=layer, method="nearest").temp.data) + 273.15 #This is potential temperature. Converted from C to K.
      oce[init][valid][layer]["s"] = float(oceData.sel(yh=point[0], xh=point[1], z_l=layer, method="nearest").so.data[0])
      oce[init][valid][layer]["u"] = float(oceData.sel(yh=point[0], xq=point[1], z_l=layer, method="nearest").uo.data[0])
      oce[init][valid][layer]["v"] = float(oceData.sel(yq=point[0], xh=point[1], z_l=layer, method="nearest").vo.data[0])

    !rm {atmFile}
    !rm {oceFile}

In [ ]:
atm

In [ ]:
oce